## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [2]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [3]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.003813645918853581
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.15939615399111062
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.008109500049613416
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.23655258293729275


## Different MatMul Algorithms

In [9]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M = 8 #4096
K = 8 #6144
N = 16 #2048
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [10]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "tiled_register")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)

building cuda graph for tiled_register
loading cuda graph...
test run:
 [[ 1.7856936   2.6822402  -0.23122573  3.134018   -0.9712914  -0.01179474
   2.5616312  -1.5849617   1.5723388  -0.02167607 -0.5903712   1.1065731
   2.069689   -0.7438354   2.5055997  -1.7748374 ]
 [ 0.6675001  -0.10622041  0.5263718  -1.3680663  -1.1336864   1.6538651
   0.63758975 -0.12662111  3.2440948   0.58786863  2.4509554   0.5248291
   4.703652   -4.1186056  -0.09473697 -3.0912485 ]
 [-0.26149082 -2.2347524   0.34930643 -0.5527227  -1.3494284   1.6126791
  -3.5710514   0.3574884   0.26646754 -0.20448758  0.34322885 -1.8933692
  -0.3826628  -0.43462113 -4.356668    0.6907728 ]
 [ 1.0615041   1.8228922  -0.15518983  1.6855638  -0.36220443 -0.18158318
   2.0154932  -0.91423786  1.0283276   0.04856261 -0.22146295  0.9457167
   1.4906869  -0.58420265  2.0862226  -1.2817472 ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0. 

AssertionError: 

In [ ]:
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The tiled matmul process looks like this:
![](tile.png)

The *tiled_register* version is further compute results in "subtiles":
![](subtile.png)

Run a complete benchmark for different algorithms:

In [140]:
import time
for algo in ["naive", "coalescing", "tiled"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.013435979800124186, 'mojo': 0.5127615508004966}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013407805200404254, 'mojo': 0.13278580000041984}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.01342437559942482, 'mojo': 0.10924591420043725}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul